## Scrapping C-Zone and Brightstar Online Store

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import datetime

In [ ]:
pd.set_option('display.max_rows', 60) # Default 60, otherwise None (to see all)

In [7]:
url = 'https://brightstarcomp.com/collections/pc-components?sort_by=title-ascending&page=1&filter.v.price.gte=&filter.v.price.lte=&filter.p.product_type=Processor'
page = requests.get(url)
soup = BeautifulSoup(page.text, 'html')

In [ ]:
print(soup)

In [ ]:
products = soup.find('div', class_='right-products')
products = products.find_all('div', class_='product-item')
print(products[0])

In [70]:
def extract_product_details(product):
    """
    Extracts the URL, name, and price from a product item.

    Args:
        product (BeautifulSoup object): A single product item.

    Returns:
        dict: A dictionary containing the product's URL, name, and price.
    """
    # Extract the component URL
    url = product.find('a')['href']
    full_url = f"https://brightstarcomp.com{url}"  # Add the base URL

    # Extract the component name
    name = product.find('p', class_='product-title').text.strip()

    # Extract the price
    price = product.find('p', class_='price').text.strip()

    # Return the extracted details as a dictionary
    return {
        'url': full_url,
        'name': name,
        'price': price,
        'timestamp': datetime.datetime.now()
    }

""" 
Extract details for all products in each category.
For: Brightstar Computer Website
As of: 19th March 2025
"""

i = 1
product_list = []
product_type = 'PC Accessories'
while(True):
    url = f'https://brightstarcomp.com/collections/pc-components?sort_by=title-ascending&page={i}&filter.v.price.gte=&filter.v.price.lte=&filter.p.product_type={product_type}'
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html')

    products = soup.find('div', class_='right-products')
    products = products.find_all('div', class_='product-item')

    if len(products) == 0:
        break
    product_list += [extract_product_details(product) for product in products]
    i += 1

# Convert the list of dictionaries into a Pandas DataFrame
df = pd.DataFrame(product_list)
df['Note'] = None

# Print the DataFrame
display(df)

# Save the DataFrame to a CSV file
df.to_csv(f'data/parts/brightstarcomp/{product_type.lower().strip()}.csv', index=False)

,url,name,price,timestamp,Note
0,https://brightstarcomp.com/collections/pc-comp...,1st Player ARGB Silicon Sleeved Extension Cabl...,RM 130.00,2025-04-02 18:59:57.131359,None
1,https://brightstarcomp.com/collections/pc-comp...,1st-player-steampunk-gold-series-psu-350mm-cable,RM 75.00,2025-04-02 18:59:57.131359,None
2,https://brightstarcomp.com/collections/pc-comp...,ADATA XPG Prime ARGB Extension For VGA Sleeved...,RM 149.00,2025-04-02 18:59:57.131359,None
3,https://brightstarcomp.com/collections/pc-comp...,Arctic A-RGB Controller with RF Remote Control...,RM 74.00,2025-04-02 18:59:57.131359,None
4,https://brightstarcomp.com/collections/pc-comp...,Arctic Case PWM Sharing Fan Hub,RM 49.00,2025-04-02 18:59:57.131359,None
...,...,...,...,...,...
62,https://brightstarcomp.com/collections/pc-comp...,Thermalright Intel LGA 1700 Air Cooler Refit B...,RM 15.00,2025-04-02 18:59:58.265344,None
63,https://brightstarcomp.com/collections/pc-comp...,Thermalright Intel LGA1700 Bending Corrector F...,RM 35.00,2025-04-02 18:59:58.265344,None
64,https://brightstarcomp.com/collections/pc-comp...,Thermaltake LCD Panel Kit for The Tower 500 - ...,RM 99.00,2025-04-02 18:59:58.265344,None
65,https://brightstarcomp.com/collections/pc-comp...,Thermaltake The Tower 300 Chassis Stand Kit - ...,RM 129.00,2025-04-02 18:59:58.265344,None


In [68]:
def extract_product_details(product):
    try:
        # Extract the component URL
        url = product.find('a', class_='product-item-link')['href']

        # Extract the component name
        name = product.find('strong', class_='product name product-item-name').text.strip()

        # Extract the price
        price_tag = product.find('span', class_='price-wrapper')
        price = price_tag['data-price-amount'] if price_tag else None  # Handle missing price

        # Return the extracted details as a dictionary
        return {
            'url': url,
            'name': name,
            'price': price,
            'timestamp': datetime.datetime.now()
        }
    except AttributeError as e:
        print(f"Error extracting product details: {e}")
        return None

In [69]:
import os

""" 
Extract details for all products in each category.
For: C-zone Website
As of: 19th March 2025
"""
components_done = ['cpu-processors', 'memory', 'motherboards','video-card-video-devices']
components = ['computer-cases', 'power-supplies', 'fan-pc-cooling', 'sound-cards', 'optical-drives']
output_dir = 'data/parts/czone/'
os.makedirs(output_dir, exist_ok=True)

for component in components:
    product_list = []
    seen_products = set()  # To track unique product URLs
    i = 1

    while True:
        # Scrape page
        url = f'https://czone.my/czone/computer-components/core-components/{component}.html?p={i}'
        page = requests.get(url)
        if page.status_code != 200:
            print(f"Failed to fetch {url}, status code: {page.status_code}")
            break

        soup = BeautifulSoup(page.text, 'html')

        # Extract data
        products_container = soup.find('div', class_='products wrapper grid products-grid')
        if not products_container:
            print(f"No products found on page {i} for component {component}")
            break

        products = products_container.findAll('li', class_='item product product-item')
        if len(products) == 0:
            break

        for product in products:
            product_details = extract_product_details(product)
            if product_details:
                product_url = product_details['url']  # Use URL as a unique identifier
                if product_url in seen_products:
                    print(f"Duplicate product detected: {product_url}. Stopping.")
                    break  # Stop processing further pages
                seen_products.add(product_url)
                product_list.append(product_details)
        else:
            # Continue to the next page if no duplicates were found
            i += 1
            continue

        # Break the outer loop if a duplicate is found
        break

    df = pd.DataFrame(product_list)
    df['Note'] = None

    # Save the DataFrame to a CSV file
    df.to_csv(f'{output_dir}{component}.csv', index=False, encoding='utf-8')

Duplicate product detected: https://czone.my/czone/nzxt-h9-computer-case.html. Stopping.
Duplicate product detected: https://czone.my/czone/mag a750bn pcie5-1739103631.html. Stopping.
Duplicate product detected: https://czone.my/czone/idc-hub-fh-07-1739103466.html. Stopping.
Duplicate product detected: https://czone.my/czone/cdkhb-1737817497.html. Stopping.
Duplicate product detected: https://czone.my/czone/mr193bk-1739103713.html. Stopping.


In [67]:
url = 'https://czone.my/czone/computer-components/core-components/computer-cases.html?p=1'
page = requests.get(url)
soup = BeautifulSoup(page.text, 'html')

products = soup.find('div', class_='products wrapper grid products-grid')
products = products.findAll('li', class_='item product product-item')
products_list = [extract_product_details(product) for product in products]
df = pd.DataFrame(products_list)

display(df)

,url,name,price,timestamp
0,https://czone.my/czone/phanteks-nv5s-computer-...,Phanteks NV5S Computer Case,449,2025-03-19 14:37:43.252984
1,https://czone.my/czone/deepcool-ch260-matx-com...,Deepcool CH260 MATX Computer Case,259,2025-03-19 14:37:43.254101
2,https://czone.my/czone/tecware-timber-m-tg-com...,Tecware Timber M TG Computer Case,179,2025-03-19 14:37:43.254101
3,https://czone.my/czone/pa401/bk/wood/tg-174192...,Asus ProArt PA401 Wood Edition Computer Case ...,560,2025-03-19 14:37:43.254101
4,https://czone.my/czone/pc-o11vpw-1739103813.html,Lian Li PC-O11 Vision Compact Computer Case,1920,2025-03-19 14:37:43.254610
5,https://czone.my/czone/twca-vxprm-bkar-1739107...,"Tecware VX Prism M Computer Case (mATX, TG, AR...",269,2025-03-19 14:37:43.254610
6,https://czone.my/czone/lian-li-a4-h2o-computer...,Lian Li A4-H2O Computer Case,679,2025-03-19 14:37:43.254610
7,https://czone.my/czone/twca-fus2a-bk-173910708...,"Tecware Fusion 2 Air Computer Case (mATX, TG, ...",229,2025-03-19 14:37:43.255006
8,https://czone.my/czone/twca-edm2-bkol-17391070...,"Tecware Edge M2 TG Computer Case (mATX, TG, AR...",169,2025-03-19 14:37:43.255006
9,https://czone.my/czone/gfg-p2storm-1739103401....,"Gaming Freak P2 Storm TG Computer Case (mATX, ...",189,2025-03-19 14:37:43.255006


In [26]:
url = f'https://brightstarcomp.com/collections/pc-components?sort_by=title-ascending&page=30&filter.v.price.gte=&filter.v.price.lte=&filter.p.product_type=Processor'
page = requests.get(url)
soup = BeautifulSoup(page.text, 'html')

products = soup.find('div', class_='right-products')
products = products.find_all('div', class_='product-item')
len(products)

0

In [23]:
df.to_csv('cpu_test.csv', index=False)

## Data Cleaning

### CPU

In [58]:
cpu = pd.read_csv('data/parts/CPU_Data.csv')
bsc_cpu = pd.read_csv('data/parts/brightstarcomp/cpu.csv')
czone_cpu = pd.read_csv('data/parts/czone/cpu-processors.csv')

In [32]:
print(cpu.columns)
print(bsc_cpu.columns)

Index(['Brand', 'Name', 'Chipset', 'Score', 'Price'], dtype='object')
Index(['url', 'name', 'price', 'timestamp', 'Note'], dtype='object')


In [72]:
def parse_cpu_details(product_string):
    """
    Parses a product string into Brand, Name, and Chipset.

    Args:
        product_string (str): The product string to parse.

    Returns:
        dict: A dictionary containing Brand, Name, and Chipset.
    """
    # Split the string into parts
    parts = product_string.split()
    
    # Extract Brand (first word)
    brand = parts[0]
    
    # Extract Chipset (if it contains "LGA" or "AM" as a standalone part)
    chipset = next((part for part in parts if part.startswith("LGA") or part.startswith("AM") and part != "AMD"), None)
    
    # Extract Name (everything between Brand and Chipset)
    if chipset:
        chipset_index = parts.index(chipset)
        name = " ".join(parts[1:chipset_index])
    else:
        name = " ".join(parts[1:])
    
    return {
        "Brand": brand,
        "Name": name,
        "Chipset": chipset
    }
# Name
name_strings = bsc_cpu['name'].tolist() + czone_cpu['name'].tolist()
new_cpu_df = pd.DataFrame([parse_cpu_details(name) for name in name_strings])
# Price
bsc_price_string = bsc_cpu['price'].tolist() 
cleaned_price_string = [price.replace('RM', '').replace(',', '').strip() for price in bsc_price_string]
price_strings = cleaned_price_string + czone_cpu['price'].tolist()
new_cpu_df['Price'] = price_strings
# URL
new_cpu_df['URL'] = bsc_cpu['url'].tolist() + czone_cpu['url'].tolist()
# Timestamp
new_cpu_df['Timestamp'] = bsc_cpu['timestamp'].tolist() + czone_cpu['timestamp'].tolist()
# Note
new_cpu_df['Note'] = None
display(new_cpu_df)

,Brand,Name,Chipset,Price,URL,Timestamp,Note
0,AMD,Athlon 3000G,AM4,219.00,https://brightstarcomp.com/collections/pc-comp...,2025-03-18 22:10:07.371057,None
1,AMD,Ryzen 3 3200G,AM4,299.00,https://brightstarcomp.com/collections/pc-comp...,2025-03-18 22:10:07.371057,None
2,AMD,Ryzen 5 5500,AM4,409.00,https://brightstarcomp.com/collections/pc-comp...,2025-03-18 22:10:07.371057,None
3,AMD,Ryzen 5 5600,AM4,499.00,https://brightstarcomp.com/collections/pc-comp...,2025-03-18 22:10:07.371057,None
4,AMD,Ryzen 5 5600G,AM4,549.00,https://brightstarcomp.com/collections/pc-comp...,2025-03-18 22:10:07.371057,None
...,...,...,...,...,...,...,...
88,Intel,Core i5 12400 Processor,None,569,https://czone.my/czone/bx8071512400-1739103180...,2025-03-19 14:21:03.565474,None
89,AMD,Ryzen 5 5600G Processor,None,499,https://czone.my/czone/100-100000252box-173910...,2025-03-19 14:21:03.565474,None
90,AMD,Ryzen 7 5700G Processor,None,809,https://czone.my/czone/100-100000263box-173910...,2025-03-19 14:21:03.565474,None
91,AMD,Ryzen 5 5600X Processor,None,629,https://czone.my/czone/100-100000065box-173910...,2025-03-19 14:21:03.566474,None


In [71]:
merged_cpu = pd.DataFrame(pd.merge(cpu, new_cpu_df, how='outer'))
# Remove Chipset blankspace
merged_cpu['Chipset'] = [str(chipset).replace(' ', '') for chipset in merged_cpu['Chipset']]
# Set null score to -1
merged_cpu['Score'] = [
    int(score) if pd.notnull(score) else -1 for score in merged_cpu['Score']
]
merged_cpu['Name'] = merged_cpu['Name'].str.replace('Processor', '')
merged_cpu = merged_cpu.where(pd.notnull(merged_cpu), None)
display(merged_cpu)

,Brand,Name,Chipset,Score,Price,URL,Timestamp,Note
0,AMD,Athlon 3000G,AM4,-1,219.00,https://brightstarcomp.com/collections/pc-comp...,2025-03-18 22:10:07.371057,None
1,AMD,Ryzen 3 1200,AM4,6282,100,None,None,None
2,AMD,Ryzen 3 3100,AM4,11587,210,None,None,None
3,AMD,Ryzen 3 3200G,AM4,7124,300,None,None,None
4,AMD,Ryzen 3 3200G,AM4,-1,299.00,https://brightstarcomp.com/collections/pc-comp...,2025-03-18 22:10:07.371057,None
...,...,...,...,...,...,...,...,...
123,Intel,Core i9-14900K,LGA1700,-1,2199.00,https://brightstarcomp.com/collections/pc-comp...,2025-03-18 22:10:08.566930,None
124,Intel,Core i9-14900KF,LGA1700,59263,2100,None,None,None
125,Intel,Core i9-14900KF,LGA1700,-1,2099.00,https://brightstarcomp.com/collections/pc-comp...,2025-03-18 22:10:08.566930,None
126,Intel,Core i9-14900KS,LGA1700,-1,3299.00,https://brightstarcomp.com/collections/pc-comp...,2025-03-18 22:10:08.566930,None


Commented out because manually cleaned (don't want to overwrite)

In [ ]:
# merged_cpu.to_csv('data/parts/CPU_Data(new).csv', index=False, encoding='utf-8')

### Case

In [100]:
pd.set_option('display.max_rows', None) # Default 60, otherwise None (to see all)

In [85]:
pc_case = pd.read_csv('data/parts/Case_Data.csv')
bsc_case = pd.read_csv('data/parts/brightstarcomp/case.csv')
czone_case = pd.read_csv('data/parts/czone/computer-cases.csv')

In [86]:
print(f'{pc_case.columns}\n{bsc_case.columns}\n{czone_case.columns}')


Index(['Brand', 'Model', 'Size', 'Color', 'Price'], dtype='object')
Index(['url', 'name', 'price', 'timestamp', 'Note'], dtype='object')
Index(['url', 'name', 'price', 'timestamp', 'Note'], dtype='object')


In [126]:
def parse_case_details(product_string):
    # Split the string into parts
    parts = product_string.split()
    brands = ['1st Player', 'Aerocool', 'Aigo', 'Antec', 'Asus', 'BeQuiet', 'Cooler Master',
              'Corsair', 'Deepcool', 'DeepCool', 'E-SUPERCASE', 'Fractal Design', 'GIGABYTE',
              'Gigabyte', 'HYTE', 'Hyte', 'Invasion', 'Lian Li', 'MAGNIUMGEAR', 'MONTECH', 'Montech', 'MSI', 
              'NZXT', 'Phanteks', 'Segotep', 'Slevcase', 'SSUPD', 'Tecware', 'Thermaltake',
              'X-Five', 'Zalman', 'Gaming Freak', 'Jonsbo', 'Silverstone']
    
    # Extract Brand (check for multi-word matches)
    brand = next((brand for brand in brands if " ".join(parts[:len(brand.split())]) == brand), None)
    
    # Extract Name (everything after Brand)
    if brand:
        brand_index = len(brand.split())  # Get the number of words in the brand
        name = " ".join(parts[brand_index:])
    else:
        name = " ".join(parts)

    # Clean name
    strings = ['Computer Case', 'PC Chassis', 'Pc Chassis', 'Chassis', 'PC Case', 'Pc Case', 'Case']
    for string in strings:
        name = name.replace(string, '').strip()
    
    # Extract extra data (content inside parentheses)
    if '(' in product_string and ')' in product_string:
        start = product_string.find('(')
        end = product_string.find(')')
        extra_data = product_string[start + 1:end]

        # Remove extra data from name
        name = name.replace(product_string[start:end+1], '').strip()
    else:
        extra_data = None

    # Extract size (if available) - ensure exact matches
    sizes = ['Micro-ATX', 'Mini-ITX', 'E-ATX','EATX', 'mATX','MATX', 'ATX', 'ITX']
    size = None
    for s in sizes:
        if f"{s}" in f"{product_string}":
            size = s
            break  # Exit the loop as soon as a match is found

    # Remove size from name if found
    if size:
        name = name.replace(size, '').strip()
    else:
        size = None

    # Extract tower type (if available)
    towers = ['Mid-Tower', 'Mid Tower', 'Full-Tower', 'Full Tower', 'Super-Tower']
    tower = next((t for t in towers if t in product_string), None)

    # Append tower type to extra data if found
    if tower:
        name = name.replace(tower, '').strip()
        if extra_data:
            extra_data += f", {tower}"
        else:
            extra_data = tower

    return {
        "Brand": brand,
        "Name": name,
        "Size": size,
        "Price": None,
        "URL": None,
        "Timestamp": None,
        "Note": extra_data
    }

case_name_strings = bsc_case['name'].tolist() + czone_case['name'].tolist()
new_case_df = pd.DataFrame([parse_case_details(name) for name in case_name_strings])
new_case_df['Price'] = bsc_case['price'].tolist() + czone_case['price'].tolist()
new_case_df['URL'] = bsc_case['url'].tolist() + czone_case['url'].tolist()
new_case_df['Timestamp'] = bsc_case['timestamp'].tolist() + czone_case['timestamp'].tolist()

display(new_case_df)

,Brand,Name,Size,Price,URL,Timestamp,Note
0,1st Player,Black Sir B5-M,Micro-ATX,RM 135.00,https://brightstarcomp.com/collections/pc-comp...,2025-03-18 22:12:10.939353,None
1,1st Player,BS-2,Micro-ATX,From RM 150.00 to RM 165.00,https://brightstarcomp.com/collections/pc-comp...,2025-03-18 22:12:10.939353,None
2,1st Player,D7,Micro-ATX,From RM 179.00 to RM 189.00,https://brightstarcomp.com/collections/pc-comp...,2025-03-18 22:12:10.939353,None
3,1st Player,Mi5,Micro-ATX,From RM 169.00 to RM 179.00,https://brightstarcomp.com/collections/pc-comp...,2025-03-18 22:12:10.940351,None
4,1st Player,Mi7-A BLACK / WHITE Double Sided Tempered Glas...,ATX,RM 199.00,https://brightstarcomp.com/collections/pc-comp...,2025-03-18 22:12:10.940351,None
5,1st Player,Mi8,Micro-ATX,From RM 354.00 to RM 369.00,https://brightstarcomp.com/collections/pc-comp...,2025-03-18 22:12:10.940351,None
6,1st Player,MV5-TP,Micro-ATX,From RM 199.00 to RM 209.00,https://brightstarcomp.com/collections/pc-comp...,2025-03-18 22:12:10.940351,None
7,1st Player,MV8-TP Tempered Glass,Micro-ATX,From RM 329.00 to RM 349.00,https://brightstarcomp.com/collections/pc-comp...,2025-03-18 22:12:10.940351,None
8,1st Player,SP7 Tempered Glass,Micro-ATX,From RM 379.00 to RM 399.00,https://brightstarcomp.com/collections/pc-comp...,2025-03-18 22:12:10.940351,None
9,1st Player,T3 Tempered Glass,Micro-ATX,From RM 185.00 to RM 195.00,https://brightstarcomp.com/collections/pc-comp...,2025-03-18 22:12:10.940351,None


In [128]:
new_case_df.to_csv('data/parts/Case_Data(new-excl-old).csv', index=False, encoding='utf-8')